## Import required modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ase
from ase import Atom, Atoms, cluster, units
from ase.visualize import view
from NCMD import create,simulate, analyze

## Create Species to add to scene

In [ ]:
cell_dimension = 500
cell = np.array([[cell_dimension,0,0],[0,cell_dimension,0,],[0,0,cell_dimension]])
a = 4.07 #Lattice parameter Au

ico = ase.cluster.Icosahedron(symbol = 'Au', noshells = 2)
dec = ase.cluster.Decahedron(symbol = 'Au', p = 1, q = 1, r = 1)
tet_coords = [(0,0,0) , (0,a/2,a/2) , (a/2,a/2,0) , (a/2,0,a/2)]
Au_tet = Atoms('Au4', positions = tet_coords)
Au_pair = Atoms('Au2', positions = [(0,0,0), (0,0,a)])
Au = Atoms('Au', positions = [(0,0,0)])

In [ ]:
ico.set_cell(cell)
dec.set_cell(cell)
Au_tet.set_cell(cell)
Au_pair.set_cell(cell)
Au.set_cell(cell)

## Create Scene and set simulation cell

In [ ]:
atoms0 = create.create_scene(ico, ico, 54, 38)
atoms1 = create.create_scene(atoms0, Au_tet, 40, 65)
atoms2 = create.create_scene(atoms1, dec, 40, 20)
atoms3 = create.create_scene(atoms2, Au_pair, 20, 56)
atoms = create.create_scene(atoms3, Au, 20, 10)

In [ ]:
##Checks the information within the atoms object to ensure that the number of atoms is appropriate

atoms

In [ ]:
atoms.set_cell(cell)
atoms.center()

## Check the created scene to ensure that the randomly allocated species were appropriately distributed

In [ ]:
dists = atoms.get_all_distances()

In [ ]:
#dists*=100
dists_max = np.max(dists)
dists_min = np.min(dists)
print(dists_max)

In [ ]:
dist_counts, dist_bins = analyze.get_radial_distance(atoms, np.arange(0,600,0.1))

In [ ]:
fig, ax = plt.subplots()
ax.plot(dist_bins, dist_counts)
ax.set_xlabel('Distance (Angstroms)', size = 15)
ax.set_ylabel('Counts', size = 15)
plt.show()

In [ ]:
view(atoms)

In [ ]:
atoms.set_pbc(True)
atoms.get_pbc()

## Write out scene for MD_simulate.ipynb

In [ ]:
scene_name = 'test_scene.xyz'
ase.io.write(scene_name, atoms)

## Simulate (import relevant data if skipping scene builder)

In [ ]:
# Attach a Lennard-Jones pair-potential
sigma = 0.2651 * units.nm  # 1
epsilon = 2567*units.kB    # 1
r_cutoff = 3*sigma
r_onset = 2/3*r_cutoff

In [ ]:
r = np.arange(0.0001,10,step=0.0001)
lj_pot = (4 * epsilon) * ( (sigma/r)**12 - (sigma/r)**6 )
lj_f = -(24 * epsilon * sigma**6) *( (1/r**7) - (2*sigma**6 / r**13))

In [ ]:
fig, ax = plt.subplots(figsize = (6,5))
ax.plot(r,lj_pot, label = 'Potential', color = 'r')
ax.plot(r, lj_f, label = 'Force', color = 'b')

ax.set_ylim(-0.4,0.6)
ax.set_xlim(2,8)
ax.vlines((r_onset),ax.get_ylim()[0],ax.get_ylim()[1],ls=':', color = 'k', label = 'Cutoff')
ax.set_xlabel('r')
ax.set_ylabel('V(r) or F(r)')
plt.legend()

plt.show()

In [ ]:
scene_name = 'test_scene.xyz'
in_dat = ase.io.read(filename = scene_name)
in_dat.set_cell(cell)
in_dat.center()
in_dat.set_pbc(True)
atoms = in_dat

In [ ]:
atoms.set_pbc([True, True, True])
atoms.pbc

In [ ]:
calc = lj.LennardJones()
calc.parameters

In [ ]:
calc.parameters['epsilon'] = epsilon
calc.parameters['sigma'] = sigma
calc.parameters['rc'] = r_cutoff
calc.parameters['ro'] = r_onset

In [ ]:
calc.parameters

In [ ]:
calc = LennardJones(atoms.arrays['numbers'][0], sigma=sigma, epsilon=epsilon, rCut=r_cutoff, modified=True)

In [ ]:
atoms.calc = calc
fn = 'test_simulation.traj'

In [ ]:
atoms_final = sim.simulate(atoms, fn = fn, temp = 100, 
                           time_step = 5, length = 1000, friction =0.005,
                          writeout = 100)

In [ ]:
traj2 = ase.io.read(fn, index='1:5000')
view(traj2, viewer='ngl')